In [ ]:
# 导入一些必要的库
import numpy as np
import h5py
import matplotlib.pyplot as plt
from lr_utils import load_dataset

In [ ]:
train_set_x_orig, train_set_y, test_set_x_orig, test_set_y, classes = load_dataset()  #加载训练、测试数据

# 看看训练数据样子
print('x.shape = {}\ny.shape = {}\nclasses = {}'.format(train_set_x_orig.shape, train_set_y.shape, classes)) #看看数据是什么大小

index = 25
plt.imshow(train_set_x_orig[index])  # 把第25张图片打印出来看看
print(train_set_y[0, 1:5])
ind = np.squeeze(train_set_y[0,index]) #获得第25张图片对应的标签值，squeeze是去掉矩阵的方括号的作用
print("The picture's label is y = {},and it's a {} picture".format(train_set_y[0, index], classes[ind].decode('utf-8')))

In [ ]:
# 处理数据，使得：1.图片集成为一个矩阵，该矩阵每列代表一张图片，列数代表图片数量 2.标签成为行向量，每列数字是对应的标签值
x_train_num = train_set_x_orig.shape[0]
x_train_h = train_set_x_orig.shape[1]
x_train_w = train_set_x_orig.shape[2]
x_train_c = train_set_x_orig.shape[3]
x_train_size = x_train_h*x_train_w*x_train_c

x_test_shape = test_set_x_orig.shape
x_test_num = test_shape[0]
x_test_h = test_shape[1]
x_test_w = test_shape[2]
x_test_c = test_shape[3]
x_test_size = x_test_h*x_test_w*x_test_c

x_train_flatten = train_set_x_orig.reshape((x_train_num, -1)).T  #将图片压扁成一维行向量,每列是一张图片,也可以这样写，-1表示可以计算这个维度，不用显式表示
x_test_flatten = test_set_x_orig.reshape((test_num, -1)).T  
x_train_flatten = x_train_flatten / 255  #像素值压缩到0到1之间
x_test_flatten = x_test_flatten / 255

y_train = train_set_y.reshape((1, train_set_y.shape[1])).T  #列向量
y_test = test_set_y.reshape((1, test_set_y.shape[1])).T

print(x_train_flatten.shape)

assert(x_train_flatten.shape == (x_train_size, x_train_num))

In [ ]:
def sigmoid(z):
    s = 1 / (1 + np.exp(-z))
    return s

In [ ]:
def initialize(dim):
    w = np.zeros(shape = (dim, 1))
    b = 0
    assert(w.shape == (dim, 1))
    assert(isinstance(b, int) or isinstance(b, float))
    return (w, b)
    

In [ ]:
# 前向传播，计算模型值，计算当前参数预测的loss
def Loss(x, w, b, y):  #x一列是一张图片，w是列向量， y是列向量
    m = x.shape[1]
    h_x = sigmoid(np.dot(x.T, w) + b)
    #print('*******',h_x)
    cost = -1/m * (np.dot(y.T, np.log(h_x)) + np.dot(1 - y.T, np.log(1 - h_x)))
    # cost = (- 1 / m) * np.sum(y * np.log(h_x) + (1 - y) * (np.log(1 - h_x)))
    # print('{}\n'.format(h_x[1:10]))
    #print(y.T.shape)
    #print(h_x.shape)
    cost = np.squeeze(cost)
    return (h_x, cost)

In [ ]:
w, b, X, Y = np.array([[1], [2]]), 2, np.array([[1,2], [3,4]]), np.array([[1, 0]]).T
h_x, cost = Loss(X,w, b, Y)
print(cost)

In [ ]:
# 计算参数偏导数
def delat(x, w, b, y): #h_x是列向量，y是列向量
    #print('{}x.shape = {}'.format('*-*'*10,x.shape))
    #print('{}w.shape = {}'.format('*-*'*10,w.shape))
    m =x.shape[1]
    h_x, _ = Loss(x, w, b, y)
    dz = h_x - y
    #print('{}dz.shape = {}'.format('*-*'*10,dz.shape))
    assert(dz.shape == (m, 1))
    dw = 1/m * np.dot(x, dz)
    db = 1/m * np.sum(dz)
    delat_parameter = {'delta_w':dw, 'delta_b':db}
    return delat_parameter  

In [ ]:
w, b, X, Y = np.array([[1], [2]]), 2, np.array([[1,2], [3,4]]), np.array([[1, 0]])
delat_p = delat(X,w,b,Y.T )
dw = delat_p['delta_w']
print(dw.shape)
db = delat_p['delta_b']
print('dw = \n{}\ndb = {}'.format(dw, db))

In [ ]:
#梯度下降
def train(x, y, num_iterations, rate, initial_w, initial_b, print_loss = False):
    n = x.shape[0]
    w = initial_w
    b = initial_b
    cost = []
    print('{}Start training{}'.format('='*20, '='*20))
    print('The times of iteration is :',num_iterations)
    for i in range(num_iterations):  
        _, loss = Loss(x, w, b, y)       
        delat_parameter = delat(x, w, b, y)
        dw = delat_parameter['delta_w']
        db = delat_parameter['delta_b']
        w = w - rate * dw
        b = b - rate * db
        
        if i % 100 == 0:
            cost.append(loss)

        if print_loss and i % 100 == 0:
            print('Iteration ：{} | loss : {}'.format(i, loss))
    print('{}training end{}'.format('='*20, '='*20))        
    param = {'w':w, 'b':b}
    gradient = {'delat_w':dw, 'delat_b':db}
    assert(w.shape == (n,1))
    return (param, gradient, cost)

In [ ]:
#测试梯度下降
w = np.array([1,2])
w_in = w.reshape((2, 1))

b = 2
y = np.array([1,0])
y_t = y.reshape((2, 1))

x = np.array([1, 2, 3, 4])
x_t = x.reshape((2, 2))

param, gradient, _ = train(x_t, y_t, 100, 0.009, w_in, b, False)
print('w = {}\nb = {}\ndw = {}\ndb = {}'.format(param['w'], param['b'],gradient['delat_w'], gradient['delat_b']))

In [ ]:
def predict(x, w, b):
    #print(x.shape)
    m = x.shape[1]
    pre = np.zeros((m, 1))
    z = np.dot(x.T, w) + b
    a = sigmoid(z)
    for i in range(a.shape[0]):
        if a[i, 0] > 0.5: 
            pre[i, 0] = 1
        else:
            pre[i, 0] = 0
    assert(pre.shape == (m, 1))
    return pre
    

In [ ]:
# 定义一个大函数，传入训练集、测试集以及训练参数，返回：最终优化的参数值、训练集、测试集上的准确率
def model(train_x, train_y, test_x, test_y, num_iterations, rate, print_cost):
    initial_w, initial_b = initialize(train_x.shape[0])
    param, _ , cost= train(train_x, train_y, num_iterations, rate, initial_w, initial_b, print_cost)
    w = param['w']
    b = param['b']
    
    train_pre = predict(train_x, w, b)
    test_pre = predict(test_x, w, b)
    
    accuracy_train_data = str((1 - np.mean(np.abs(train_pre - train_y))) * 100) + '%'
    accuracy_test_data = str(1 - np.mean(np.abs(test_pre - test_y))) + '%'
    
    print('The accuracy in tain data is {}\nThe accuracy in test data is {}\n'.format(accuracy_train_data, accuracy_test_data))
    
    result = {'w':w, 'b':b, 'train_pre':train_pre, 'test_pre':test_pre, 'los':cost, 'rate':rate}
    return result    

In [ ]:
print('{}Testing model{}'.format('='*20, '='*20))
print('x.shape = {} \ny.shape = {}'.format(x_train_flatten.shape,  y_train.shape))
result = model(x_train_flatten, y_train, x_test_flatten, y_test, num_iterations = 2000, rate = 0.005, print_cost = True)

In [ ]:
#绘制相关图
loss = np.squeeze(result['los'])
plt.plot(loss)
plt.ylabel('cost')
plt.xlabel('iterations')
plt.title('Learning rate = {}'.format(result['rate']))
plt.show()

In [ ]:
learning_rate = [0.01, 0.001, 0.0001]
results = { } #空字典,装3个model返回的结果
for r in learning_rate:
    print('The learning rate is : ', r)
    results[str(r)] = model(x_train_flatten, y_train, x_test_flatten, y_test, num_iterations = 2000, rate = r, print_cost = False)
    print('{} 这是分割线{}\n '.format('!*--*! +'*5, '!*--*! +'*5))
    
for i in learning_rate:
    plt.plot(np.squeeze(results[str(i)]['los']), label= str(results[str(i)]['rate']))

plt.xlabel('iterations')
plt.ylabel('loss')

legend = plt.legend(loc = 'upper right', shadow = True) #图例
legend.get_frame().set_facecolor('0.50')  #定义图例框背景颜色深浅
legend.get_frame().set_linewidth(0)
plt.show()
